In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


### Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksetenimit.dfs.core.windows.net/customers")

In [0]:
display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
 
df = df.withColumn('Domains', split(col('email'), '@')[1])
display(df)

In [0]:
df.groupBy("Domains").agg(count(col("customer_id")).alias("count")).sort(col("count").desc()).display()


In [0]:
df_gmail = df.filter(df.Domains == "gmail.com")
display(df_gmail)

df_yahoo = df.filter(df.Domains == "yahoo.com")
display(df_yahoo)

df_hotmail = df.filter(df.Domains == "hotmail.com")
display(df_hotmail)


In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop('first_name','last_name')
display(df)

In [0]:
df.write.mode("append").format("delta").save("abfss://silver@databricksetenimit.dfs.core.windows.net/customers")